In [0]:
%run ./1-config

In [0]:
class SetupHelper():   
    def __init__(self):
        Conf = Config()
        self.landing_zone = Conf.base_dir_data + "/raw"
        self.checkpoint_base = Conf.base_dir_checkpoint + "/checkpoints"        
        self.db_name = Conf.db_name
        self.initialized = False
        
    def create_db(self):
        spark.catalog.clearCache()
        print(f"Creating the database {self.db_name}...", end='')
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.db_name}")
        spark.sql(f"USE {self.db_name}")
        self.initialized = True
        print("Done")


    def create_kafka_multiplex_bz(self,table_name):
        if(self.initialized):
            print(f"Creating {table_name} table...", end='')
            spark.sql(f"""CREATE TABLE IF NOT EXISTS {self.db_name}.{table_name}(
                  key string, 
                  value string, 
                  topic string, 
                  partition bigint, 
                  offset bigint, 
                  timestamp bigint,                  
                  timestamp_type integer)
                  """) 
            print("Done")
        else:
            raise ReferenceError("Application database is not defined. Cannot create table in default database.")    


    def setup(self):
        if not self.initialized:
            self.create_db()
        self.create_kafka_multiplex_bz("registered_users_tables")  
        self.create_kafka_multiplex_bz("gym_logins_table")  
        self.create_kafka_multiplex_bz("bpm_stream_table") 
        self.create_kafka_multiplex_bz("workout_session_table")  
        self.create_kafka_multiplex_bz("user_info_table")   

In [0]:
sh = SetupHelper()

In [0]:
sh.setup()

Creating the database sbit_db...Done
Creating kafka_multiplex_bz table...Done
Creating kafka_multiplex_bz table...Done
Creating kafka_multiplex_bz table...Done
Creating kafka_multiplex_bz table...Done
Creating kafka_multiplex_bz table...Done


In [0]:
from pyspark.sql import SparkSession

# create SparkSession
spark = SparkSession.builder.appName("TableList").getOrCreate()

# list of table names in a database
databaseName = "sbit_db"
tableNames = spark.catalog.listTables(databaseName)

# print table names
for tableName in tableNames:
    print(tableName.name)

# stop SparkSession


bpm_stream_table
gym_logins_table
registered_users_tables
user_info_table
workout_session_table
